# Imports

In [ ]:
!pip install tensorflow==2.15

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import os
import time
import pathlib


from matplotlib import pyplot as plt
from IPython import display


# Pix2Pix

Pix to Pix is a technique for image translation using conditional GANs. In this way we can colorize black and white photos, convert google maps to google earth, etc.

We are going to convert building facades drawing to real buildings.

In example, we will use the [CMP Facade Database](http://cmp.felk.cvut.cz/~tylecr1/facade/), helpfully provided by the [Center for Machine Perception](http://cmp.felk.cvut.cz/) at the [Czech Technical University in Prague](https://www.cvut.cz/).

We will use a preprocessed [copy](https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/) of this dataset, created by the authors of the [paper](https://arxiv.org/abs/1611.07004) above.

Each epoch takes around 15 seconds on a single V100 GPU.

Below is the output generated after training the model for 200 epochs.

![sample output_1](https://www.tensorflow.org/images/gan/pix2pix_1.png)
![sample output_2](https://www.tensorflow.org/images/gan/pix2pix_2.png)

# Load the dataset
 the dataset used in this tutorial and similar ones can be downloaded from [this page](https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/):
* cityscapes.tar.gz	 
* edges2handbags.tar.gz	
* edges2shoes.tar.gz	 
* facades.tar.gz	
* maps.tar.gz 

We apply random jittering and mirroring to the training dataset:
  * random jittering: image is resized to 286 x 286 then randomly cropped to 256x256
 * random mirroring: image randomly flipped horizontally i.e. left to right

In [ ]:
dataset_name = "facades" #@param ["cityscapes", "edges2handbags", "edges2shoes", "facades", "maps", "night2day"]


In [ ]:
_URL = f'http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/{dataset_name}.tar.gz'

path_to_zip = tf.keras.utils.get_file(
    fname=f"{dataset_name}.tar.gz",
    origin=_URL,
    extract=True)

path_to_zip  = pathlib.Path(path_to_zip)


PATH = path_to_zip.parent/dataset_name

Inspect one image

In [ ]:
sample_image = tf.io.read_file(os.path.join(PATH,'train/1.jpg'))
sample_image = tf.io.decode_jpeg(sample_image)
print(sample_image.shape)
plt.figure(figsize=(20,10))
plt.imshow(sample_image,aspect='auto')

In [ ]:
# Params
BUFFER_SIZE = 400
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256

In [ ]:
# Load and split image between real and input
def load(image_file):
  image = tf.io.read_file(image_file)
  image = tf.image.decode_jpeg(image)

  w = tf.shape(image)[1]

  w = w // 2
  real_image = image[:, :w, :]
  input_image = image[:, w:, :]

  input_image = tf.cast(input_image, tf.float32)
  real_image = tf.cast(real_image, tf.float32)

  return input_image, real_image

In [ ]:
# Show images
inp, re = load(os.path.join(PATH,'train/120.jpg'))
# casting to int for matplotlib to show the image
plt.figure(figsize=(20,10))
plt.subplot(121)
plt.imshow(inp/255.0, aspect='auto')
plt.title('Input',fontdict={'fontsize':20})
plt.subplot(122)
plt.imshow(re/255.0, aspect='auto')
plt.title('Real',fontdict={'fontsize':20})

# Functions

In [ ]:
def resize(input_image, real_image, height, width):
  input_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  real_image = tf.image.resize(real_image, [height, width],
                               method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  return input_image, real_image

In [ ]:
def random_crop(input_image, real_image):
  stacked_image = tf.stack([input_image, real_image], axis=0)
  cropped_image = tf.image.random_crop(
      stacked_image, size=[2, IMG_HEIGHT, IMG_WIDTH, 3])

  return cropped_image[0], cropped_image[1]

In [ ]:
# normalizing the images to [-1, 1]
def normalize(input_image, real_image):
  input_image = (input_image / 127.5) - 1
  real_image = (real_image / 127.5) - 1

  return input_image, real_image

In [ ]:
@tf.function()
def random_jitter(input_image, real_image):
  # resizing to 286 x 286 x 3
  input_image, real_image = resize(input_image, real_image, 286, 286)

  # randomly cropping to 256 x 256 x 3
  input_image, real_image = random_crop(input_image, real_image)

  if tf.random.uniform(()) > 0.5:
    # random mirroring
    input_image = tf.image.flip_left_right(input_image)
    real_image = tf.image.flip_left_right(real_image)

  return input_image, real_image

Random jittering does the following:
* Resize an image to a bigger height and width
* Randomly crop to target size
* Randomly flip the image horizontally

In [ ]:
plt.figure(figsize=(6, 6))
for i in range(4):
  rj_inp, rj_re = random_jitter(inp, re)
  plt.subplot(2, 2, i+1)
  plt.imshow(rj_inp/255.0)
  plt.axis('off')
plt.show()

In [ ]:
def load_image_train(image_file):
  input_image, real_image = load(image_file)
  input_image, real_image = random_jitter(input_image, real_image)
  input_image, real_image = normalize(input_image, real_image)

  return input_image, real_image

In [ ]:
def load_image_test(image_file):
  input_image, real_image = load(image_file)
  input_image, real_image = resize(input_image, real_image,
                                   IMG_HEIGHT, IMG_WIDTH)
  input_image, real_image = normalize(input_image, real_image)

  return input_image, real_image

# Input Pipeline

In this example, we will use the [`tf.data`](https://www.tensorflow.org/guide/data) API, while it is not necessary to understand everything for the sake of this tutorial, check the link if you want to know more about how it works.

In a nutshell it is an API which allows you to handle data and feed them into tensorflow models.

In [ ]:
train_dataset = tf.data.Dataset.list_files(os.path.join(PATH,'train/*.jpg'))
train_dataset = train_dataset.map(load_image_train,
                                  num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE)

In [ ]:
test_dataset = tf.data.Dataset.list_files(os.path.join(PATH,'test/*.jpg'))
test_dataset = test_dataset.map(load_image_test)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
test_dataset

# Build the generator
  * Architecture is a modified [U-Net](https://arxiv.org/abs/1505.04597) 
  
  (You can find out more about the implementation [here](https://www.tensorflow.org/tutorials/images/segmentation))
  * Each block in the encoder is (Conv -> [Batchnorm](https://arxiv.org/abs/1502.03167) -> [Leaky ReLU](https://ai.stanford.edu/~amaas/papers/relu_hybrid_icml2013_final.pdf))
  * There are [skip connections](https://en.wikipedia.org/wiki/Residual_neural_network) between the encoder and decoder (as in U-Net).


  ![unet_architecture](https://machinelearningmastery.com/wp-content/uploads/2019/05/Architecture-of-the-U-Net-Generator-Model.png)
  

In [ ]:
OUTPUT_CHANNELS = 3 # RGB

## Downsampling layer: FILL THE CODE

In [ ]:
#FILL THE CODE: complete the downsampling layer that we will use to shrink the image input

def downsample(filters, size, apply_batchnorm=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  #FILL THE CODE: Add a 2D convolutional layer, with the specified options, use strides=2, padding="same" and use_bias=False
  result.add(
      #...... TODO.......
  )
  if apply_batchnorm:
    result.add(tf.keras.layers.BatchNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

In [ ]:
down_model = downsample(3, 4)
down_result = down_model(tf.expand_dims(inp, 0))
print (down_result.shape)

plt.figure()
plt.subplot(121)
plt.imshow(inp, aspect='auto')
plt.subplot(122)
plt.imshow(down_result[0], aspect='auto')

## Upsampling Layer: FILL THE CODE

To upsample the images we are going to use a [`Conv2DTranspose`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) layer, which performs a [transposed convolution](https://towardsdatascience.com/what-is-transposed-convolutional-layer-40e5e6e31c11#:~:text=Transposed%20convolutions%20are%20standard%20convolutions,in%20a%20standard%20convolution%20operation.)

In [ ]:
#FILL THE CODE: complete the upsampling layer that we will increase image size after downsampling it

def upsample(filters, size, apply_dropout=False):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  #FILL THE CODE: Add a 2D transposed convolutional layer, with the specified options, use strides=2, padding='same' and use_bias=False 

  result.add(
      # ...Conv2DTranspose layer
      # ... TODO....
  )

  result.add(tf.keras.layers.BatchNormalization())

  if apply_dropout:
      result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result

In [ ]:
up_model = upsample(3, 4)
up_result = up_model(down_result)
print (up_result.shape)

plt.figure()
plt.subplot(121)
plt.imshow(down_result[0], aspect='auto')
plt.subplot(122)
plt.imshow(up_result[0], aspect='auto')

## Build the Generator by combining downsampling and upsampling layers and implementing the  skip connections

In [ ]:
def Generator():
  inputs = tf.keras.layers.Input(shape=[256,256,3])

  # Downsampling layers
  down_stack = [
    downsample(64, 4, apply_batchnorm=False), # (bs, 128, 128, 64)
    downsample(128, 4), # (bs, 64, 64, 128)
    downsample(256, 4), # (bs, 32, 32, 256)
    downsample(512, 4), # (bs, 16, 16, 512)
    downsample(512, 4), # (bs, 8, 8, 512)
    downsample(512, 4), # (bs, 4, 4, 512)
    downsample(512, 4), # (bs, 2, 2, 512)
    downsample(512, 4), # (bs, 1, 1, 512)
  ]

  # Upsampling layers
  up_stack = [
    upsample(512, 4, apply_dropout=True), # (bs, 2, 2, 1024)
    upsample(512, 4, apply_dropout=True), # (bs, 4, 4, 1024)
    upsample(512, 4, apply_dropout=True), # (bs, 8, 8, 1024)
    upsample(512, 4), # (bs, 16, 16, 1024)
    upsample(256, 4), # (bs, 32, 32, 512)
    upsample(128, 4), # (bs, 64, 64, 256)
    upsample(64, 4), # (bs, 128, 128, 128)
  ]

  initializer = tf.random_normal_initializer(0., 0.02)

  # Last layer
  last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh') # (bs, 256, 256, 3)



  # FILL THE CODE
  # Considering x as the input, try to build the model such that:
  # For the Encoder layers you simply apply the downsampling layers
  # For the Decoder layers:
  # the nth layer of the decoder takes as input the concatenation (tf.keras.layers.Concatenate) of the output of the n-1th decoder layer and n-num_decoder_layers +1 encoder layer
  # e.g. the last layer of the decoder gets as input the output of the previous layer concatenated with the output of the first encoder convolutional layer (!!not the input layer)
  
  # Downsampling through the model
  skips = []
  for down in down_stack:
    x = # FILL THE CODE
    skips.#... TODO ...

  skips = #... TODO ...

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = #... TODO ...
    x = #... TODO ...

  x = #... TODO ...

  return tf.keras.Model(inputs=inputs, outputs=x)


In [ ]:
generator = Generator()
tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)

In [ ]:
gen_output = generator(inp[tf.newaxis,...], training=False) # N.B. inp is input image
plt.imshow(gen_output[0,...])

   ## Generator Loss: FILL THE CODE
    * sigmoid cross entropy loss of generated images and and array of ones (classification)
    * L1 loss -> Mean Absolute Error (MAE) between generated image and target image
      * this allows generated image to be structurally similar to target image
      * Formula to calculate total generator loss is:
        * $ \mathcal{L}_\text{gan} +\lambda \mathcal{L}_\text{L1} $ 
        
        with $\lambda = 100$ (decided by authors of the original paper)
     

```
# Questo è formattato come codice
```


The training procedure for the generator is shown below:



In [ ]:
LAMBDA = 100


In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def generator_loss(disc_generated_output, gen_output, target):
  # FILL THE CODE:
  # define the GAN generator loss i.e. does the discriminator classify generator 
  # outputs as real?
  # HINT: use loss_object to compute the loss and tf.ones_like() to represent classification
  # result

  #gan_loss =  ...TODO...

  # FILL THE CODE: mean absolute error
  # compute mean absolute error between target image and generated image
  # HINT: use tf.reduce_mean and tf.abs

  #l1_loss =  ...TODO...

  # FILL THE CODE: compute generator loss as the sum of GAN loss and weighted
  # L1 loss

  #total_gen_loss =  ...TODO...

  return total_gen_loss, gan_loss, l1_loss

![Generator Update Image](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/gen.png?raw=1)

# Build the Discriminator
Discriminator is a PatchGAN
  * Each block is (Conv -> BatchNorm -> LeakyRelu)
  * shape of output after last layer is (bs,30,30,1)
    * N.B.-> each 30x30 patch of the output classifies a 70x70 portion of the input (PatchGAN)
  * Discriminator receives two inputs.
    * Input_1 image and target image, which it should classify as **real**.
    * Input_2 image and generated image, which it should classify as **fake**.
    * We concatenate Input_1 and Input_2 together in the code (```# tf.concat([inp, tar], axis=-1)```)




In [ ]:
def Discriminator():
  initializer = tf.random_normal_initializer(0., 0.02)

  inp = tf.keras.layers.Input(shape=[256, 256, 3], name='input_image')
  tar = tf.keras.layers.Input(shape=[256, 256, 3], name='target_image')

  x = tf.keras.layers.concatenate([inp, tar]) # (bs, 256, 256, channels*2)

  down1 = downsample(64, 4, False)(x) # (bs, 128, 128, 64)
  down2 = downsample(128, 4)(down1) # (bs, 64, 64, 128)
  down3 = downsample(256, 4)(down2) # (bs, 32, 32, 256)

  zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3) # (bs, 34, 34, 256)
  conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                kernel_initializer=initializer,
                                use_bias=False)(zero_pad1) # (bs, 31, 31, 512)

  batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

  leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

  zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu) # (bs, 33, 33, 512)

  last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                kernel_initializer=initializer)(zero_pad2) # (bs, 30, 30, 1)

  return tf.keras.Model(inputs=[inp, tar], outputs=last)

In [ ]:
discriminator = Discriminator()
tf.keras.utils.plot_model(discriminator, show_shapes=True, dpi=64)

In [ ]:
disc_out = discriminator([inp[tf.newaxis,...], gen_output], training=False)
plt.imshow(disc_out[0,...,-1], vmin=-20, vmax=20, cmap='RdBu_r')
plt.colorbar()

## Discriminator Loss: FILL THE CODE
The discriminator loss function takes 2 inputs:
* real images
  * real_loss is a sigmoid cross entropy loss of the real images and an array of ones(since these are the real images)
* generated images
  * generated_loss is a sigmoid cross entropy loss of the generated images and an array of zeros(since these are the fake images)
* Total_loss is a sum of real_loss and generated_loss

In [ ]:
def discriminator_loss(disc_real_output, disc_generated_output):
  # FILL THE CODE:
  # define the GAN discriminator loss remember ideally discriminator should classify
  # as 0 fake images and as 1 real images
  # HINT: use loss_object and tf.ones_like and tf.zeros_like

  # loss related to real images
  #real_loss = # ...TODO...


  # loss related to fake images
  #generated_loss = # ...TODO...

  # FILL THE CODE:
  # compute output loss as the sum of real and generated loss

  #total_disc_loss = # ...TODO...

  return total_disc_loss

# Define the Optimizers and Checkpoint-saver

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

# Generate Images

Write a function to plot some images during training:
  * We pass images from the test dataset to the generator
  * The generator will then translate the input image into the output
  * Last step is to plot the predictions

In [ ]:
def generate_images(model, test_input, tar):
  prediction = model(test_input, training=True)
  plt.figure(figsize=(15,15))

  display_list = [test_input[0], tar[0], prediction[0]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']

  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()

In [ ]:
for example_input, example_target in test_dataset.take(1):
  generate_images(generator, example_input, example_target)

# Training: FILL THE CODE
* For each example input generate an output
* The discriminator receives the input_image and the generated image as the first input. The second input is the input_image and the target_image.
* Calculate Generator_loss and discriminator_loss
* Then, we calculate the gradients of loss with respect to both the generator and the discriminator variables(inputs) and apply those to the optimizer.
* Then log the losses to TensorBoard.

* pay attention for training = True option for generator and discriminator


In [ ]:
EPOCHS = 5


In [ ]:
import datetime
log_dir="logs/"

summary_writer = tf.summary.create_file_writer(
  log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
@tf.function
def train_step(input_image, target, epoch):

  # Use tf.gradient tape to keep track of the generator and discriminator gradients
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

    # FILL THE code: compute the output of the generator (i.e. fake images)   
    #gen_output = # ...TODO...

    # FILL THE CODE: compute the output of the discriminator w.r.t. real images 
    #disc_real_output = # ...TODO...

    # FILL THE CODE: compute the output of the discriminator w.r.t. fake images 
    #disc_generated_output = # ...TODO...

    # compute the 2 losses
    gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  # FILL THE CODE: 
  # Compute the gradient of the generator loss w.r.t. the generator parameters 

  # HINT: gen_tape.gradient(...), disc_tape.gradient(...)  

  #generator_gradients = # ...TODO...

    # Compute the gradient of the discriminator loss w.r.t. the discriminator parameters

  #discriminator_gradients # ...TODO...

  # Apply the gradient to the optimizer so that it can update the model accordingly  
  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))

  with summary_writer.as_default():
    tf.summary.scalar('gen_total_loss', gen_total_loss, step=epoch)
    tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=epoch)
    tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=epoch)
    tf.summary.scalar('disc_loss', disc_loss, step=epoch)

Actual training loop:

* Iterates over the number of epochs.

* On each epoch it clears the display, and runs generate_images to show it's progress.
* On each epoch it iterates over the training dataset, printing a '.' for each example.
* It saves a checkpoint every 20 epochs.

In [ ]:
def fit(train_ds, epochs, test_ds):
  for epoch in range(epochs):
    start = time.time()

    display.clear_output(wait=True)

    for example_input, example_target in test_ds.take(1):
      generate_images(generator, example_input, example_target)
    print("Epoch: ", epoch)

    # Train
    for n, (input_image, target) in train_ds.enumerate():
      print('.', end='')
      if (n+1) % 100 == 0:
        print()
      train_step(input_image, target, epoch)
    print()

    # saving (checkpoint) the model every 20 epochs
    if (epoch + 1) % 20 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                        time.time()-start))
  checkpoint.save(file_prefix = checkpoint_prefix)

This training loop saves logs you can easily view in TensorBoard to monitor the training progress. Working locally you would launch a separate tensorboard process. In a notebook, if you want to monitor with TensorBoard it's easiest to launch the viewer before starting the training.

To launch the viewer paste the following into a code-cell:

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {log_dir}

Run the training loop:


In [ ]:
fit(train_dataset, EPOCHS, test_dataset)

In [ ]:
display.IFrame(
    src="https://tensorboard.dev/experiment/lZ0C6FONROaUMfjYkVyJqw",
    width="100%",
    height="1000px")

Interpreting the logs from a GAN is more subtle than a simple classification or regression model. Things to look for::

* Check that neither model has "won". If either the gen_gan_loss or the disc_loss gets very low it's an indicator that this model is dominating the other, and you are not successfully training the combined model.

* The value log(2) = 0.69 is a good reference point for these losses, as it indicates a perplexity of 2: That the discriminator is on average equally uncertain about the two options.

* For the disc_loss a value below 0.69 means the discriminator is doing better than random, on the combined set of real+generated images.

*For the gen_gan_loss a value below 0.69 means the generator i doing better than random at foolding the descriminator.

*As training progresses the gen_l1_loss should go down

# Restore the latest checkpoint and test

In [ ]:
!ls {checkpoint_dir}

In [ ]:
# Run the trained model on a few examples from the test dataset
for inp, tar in test_dataset.take(5):
  generate_images(generator, inp, tar)

##### Copyright 2019 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

